In [1]:
import os
import sys
import numpy as np
import torch
import logging

In [2]:
pkg_path = '/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN'
sys.path.insert(0, pkg_path)

In [3]:

from det3d import torchie
from det3d.torchie import Config
from det3d.datasets import build_dataset, build_dataloader
from det3d.models import build_detector
from det3d.torchie.apis import (
    get_root_logger,
    batch_processor
)
from det3d.torchie.trainer import get_dist_info, load_checkpoint
from det3d.torchie.trainer.utils import all_gather, synchronize
import pickle 
import time 

In [4]:
cfg_path = 'configs/waymo/voxelnet/two_stage/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze.py'

In [5]:
cfg = Config.fromfile(cfg_path)
cfg.local_rank = 0


In [6]:
logger = logging.getLogger()

In [7]:
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)


***************************model is build_from_cfg***************************
obj_type is str:  TwoStageDetector
[In build_from_cfg] building  <class 'det3d.models.detectors.two_stage.TwoStageDetector'>

***************************model is build_from_cfg***************************
obj_type is str:  VoxelNet
[In build_from_cfg] building  <class 'det3d.models.detectors.voxelnet.VoxelNet'>
Running super of VoxelNet (SingleStageDetector)

***************************model is build_from_cfg***************************
obj_type is str:  DynamicVoxelEncoder
[In build_from_cfg] building  <class 'det3d.models.readers.dynamic_voxel_encoder.DynamicVoxelEncoder'>

***************************model is build_from_cfg***************************
obj_type is str:  SpMiddleResNetFHD
[In build_from_cfg] building  <class 'det3d.models.backbones.scn.SpMiddleResNetFHD'>

***************************model is build_from_cfg***************************
obj_type is str:  RPN
[In build_from_cfg] building  <class 'de

In [8]:
device = torch.device('cpu')
map_location = 'cpu'
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    map_location = 'cuda:0'

In [9]:
dataset = build_dataset(cfg.data.val)


obj_type is str:  WaymoDataset
[In build_from_cfg] building  <class 'det3d.datasets.waymo.waymo.WaymoDataset'>
Using 1 sweeps
Using 196 Frames
obj_type is str:  LoadPointCloudFromFile
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.loading.LoadPointCloudFromFile'>
obj_type is str:  LoadPointCloudAnnotations
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.loading.LoadPointCloudAnnotations'>
obj_type is str:  Preprocess
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.preprocess.Preprocess'>
obj_type is str:  AssignLabel
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.preprocess.AssignLabel'>
obj_type is str:  Reformat
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.formating.Reformat'>


In [10]:
len(dataset)

196

In [11]:

data_loader = build_dataloader(
    dataset,
    batch_size=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False,
)


In [12]:
# for batch in data_loader:
#     b0 = batch
#     break

In [13]:
# b0

In [14]:
d1 = dataset.__getitem__(0)
d1

{'metadata': {'image_prefix': PosixPath('/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN/Ventoy/waymo_data/data/waymo'),
  'num_point_features': 5,
  'token': 'seq_0_frame_0.pkl'},
 'points': array([[-5.8179199e+01,  1.4228516e+00,  4.2121582e+00,  2.0786366e-01,
          1.1710643e-02],
        [-5.8116943e+01,  1.5605469e+00,  4.2102051e+00,  6.6308804e-02,
          0.0000000e+00],
        [-5.5648193e+01,  2.1669922e+00,  4.1254883e+00,  1.4640132e-01,
          2.9276608e-02],
        ...,
        [-7.7226394e-01, -8.0798416e+00,  3.3320874e-02,  4.9763549e-02,
          0.0000000e+00],
        [-7.3578119e+00, -7.9097219e+00, -6.7060620e-02,  6.2418748e-02,
          3.2204267e-01],
        [-2.3848801e+00,  2.5169072e+00, -8.1437826e-03,  3.4410238e-02,
          0.0000000e+00]], dtype=float32)}

In [15]:
def get_lidar(lidar_path):
    lidar = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 5)
    lidar[:, 3] = np.tanh(lidar[:, 3])
    return lidar
_data_path2 = '/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN/Ventoy/waymo_data/data/waymo/kitti2/raw_data/segment-13182548552824592684_4160_250_4180_250_with_camera_label'
_lidar_path = os.path.join(_data_path2, 'velodyne', f'{0:06d}.bin')
points=get_lidar(_lidar_path)


In [16]:
d0 = dataset.__getitem__(0)
d0

{'metadata': {'image_prefix': PosixPath('/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN/Ventoy/waymo_data/data/waymo'),
  'num_point_features': 5,
  'token': 'seq_0_frame_0.pkl'},
 'points': array([[-5.8179199e+01,  1.4228516e+00,  4.2121582e+00,  2.0786366e-01,
          1.1710643e-02],
        [-5.8116943e+01,  1.5605469e+00,  4.2102051e+00,  6.6308804e-02,
          0.0000000e+00],
        [-5.5648193e+01,  2.1669922e+00,  4.1254883e+00,  1.4640132e-01,
          2.9276608e-02],
        ...,
        [-7.7226394e-01, -8.0798416e+00,  3.3320874e-02,  4.9763549e-02,
          0.0000000e+00],
        [-7.3578119e+00, -7.9097219e+00, -6.7060620e-02,  6.2418748e-02,
          3.2204267e-01],
        [-2.3848801e+00,  2.5169072e+00, -8.1437826e-03,  3.4410238e-02,
          0.0000000e+00]], dtype=float32)}

In [ ]:
d0 = dataset.__getitem__(0)
d0

{'metadata': {'image_prefix': PosixPath('/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN/Ventoy/waymo_data/data/waymo'),
  'num_point_features': 5,
  'token': 'seq_0_frame_0.pkl'},
 'points': array([[-5.8179199e+01,  1.4228516e+00,  4.2121582e+00,  2.0786366e-01,
          1.1710643e-02],
        [-5.8116943e+01,  1.5605469e+00,  4.2102051e+00,  6.6308804e-02,
          0.0000000e+00],
        [-5.5648193e+01,  2.1669922e+00,  4.1254883e+00,  1.4640132e-01,
          2.9276608e-02],
        ...,
        [-7.7226394e-01, -8.0798416e+00,  3.3320874e-02,  4.9763549e-02,
          0.0000000e+00],
        [-7.3578119e+00, -7.9097219e+00, -6.7060620e-02,  6.2418748e-02,
          3.2204267e-01],
        [-2.3848801e+00,  2.5169072e+00, -8.1437826e-03,  3.4410238e-02,
          0.0000000e+00]], dtype=float32)}

In [17]:
# d0['points'][:, 4] = 1
d0['points']

array([[-5.8179199e+01,  1.4228516e+00,  4.2121582e+00,  2.0786366e-01,
         1.1710643e-02],
       [-5.8116943e+01,  1.5605469e+00,  4.2102051e+00,  6.6308804e-02,
         0.0000000e+00],
       [-5.5648193e+01,  2.1669922e+00,  4.1254883e+00,  1.4640132e-01,
         2.9276608e-02],
       ...,
       [-7.7226394e-01, -8.0798416e+00,  3.3320874e-02,  4.9763549e-02,
         0.0000000e+00],
       [-7.3578119e+00, -7.9097219e+00, -6.7060620e-02,  6.2418748e-02,
         3.2204267e-01],
       [-2.3848801e+00,  2.5169072e+00, -8.1437826e-03,  3.4410238e-02,
         0.0000000e+00]], dtype=float32)

In [18]:
d0['metadata'] = {'image_prefix': '', 'num_point_features': 5, 'token': ''}

In [19]:
d0['points'] = [torch.Tensor(d0['points'])]
d0['metadata'] = [d0['metadata']]

In [20]:
d0

{'metadata': [{'image_prefix': '', 'num_point_features': 5, 'token': ''}],
 'points': [tensor([[-5.8179e+01,  1.4229e+00,  4.2122e+00,  2.0786e-01,  1.1711e-02],
          [-5.8117e+01,  1.5605e+00,  4.2102e+00,  6.6309e-02,  0.0000e+00],
          [-5.5648e+01,  2.1670e+00,  4.1255e+00,  1.4640e-01,  2.9277e-02],
          ...,
          [-7.7226e-01, -8.0798e+00,  3.3321e-02,  4.9764e-02,  0.0000e+00],
          [-7.3578e+00, -7.9097e+00, -6.7061e-02,  6.2419e-02,  3.2204e-01],
          [-2.3849e+00,  2.5169e+00, -8.1438e-03,  3.4410e-02,  0.0000e+00]])]}

In [21]:
ckpt = load_checkpoint(model, cfg.load_from, map_location, strict=False, logger=logger)

load_checkpoint
model loaded using LSD
model loaded with state dict


In [22]:
model = model.cuda()
model.eval()
mode = "val"

In [23]:
outputs = batch_processor(
                model, d0, train_mode=False, local_rank=cfg.local_rank,
            )

/media/rajeev-gupta/Drive250/conda_envs/new_graphrcnn/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180589158/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [24]:
outputs

[{'box3d_lidar': tensor([[-31.6345, -20.3367,   0.8829,   2.0088,   4.3790,   1.7824,   1.5184],
          [-31.8536, -17.4737,   0.8493,   1.9724,   4.4023,   1.7881,   1.4757],
          [-65.6975,   6.8053,   0.6280,   2.4116,   5.8478,   2.0786,   3.0154],
          [  8.9420,   6.7381,   0.7614,   0.8798,   0.9286,   1.6249,   1.5588],
          [ 58.0841,   9.7893,   1.5420,   2.1399,   4.5818,   1.8173,  -1.2966],
          [-60.2211,  -0.7267,   0.6207,   2.5068,   6.2779,   2.1513,   1.4573],
          [ 22.7970, -17.9667,   1.1557,   2.0562,   4.6580,   1.5700,   0.1680],
          [ 29.3849,  13.8768,   1.0639,   0.8158,   0.8579,   1.6812,  -1.3366],
          [-44.9938,  47.6096,   0.5873,   2.0329,   4.4537,   1.6681,   2.4934],
          [ 29.7384,  13.3766,   0.7562,   0.6550,   0.6947,   1.0699,  -1.5313],
          [ 31.2256,  12.7504,   1.2474,   0.9122,   1.0103,   1.9573,  -0.4832],
          [-16.7956,  20.9944,   0.7383,   2.2098,   4.9936,   1.9181,  -1.5743],
 

: 